In [1]:
%matplotlib inline

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from PIL import Image
import json
from collections import Counter

In [3]:
wikimedia_url = 'https://commons.wikimedia.org/wiki/Category:Beach_vegetation'
wikimedia_api_url = 'https://commons.wikimedia.org/w/api.php'

res = requests.get(wikimedia_url)
b = BeautifulSoup(res.content, 'html.parser')

In [4]:
thumbs = b.find_all('div', {'class': 'thumb'})
print(len(thumbs))
print(thumbs[12].find_all('a', {'class': 'image'})[0].attrs['href'].split('/')[-1])
print(thumbs[12].find_all('img')[0].attrs['srcset'].split()[0])

17
File:Sulawesi_Panua_dune_trsr_DSCN0853_v1.JPG
https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Sulawesi_Panua_dune_trsr_DSCN0853_v1.JPG/180px-Sulawesi_Panua_dune_trsr_DSCN0853_v1.JPG


In [5]:
iiprop = [
#     'timestamp',
    'url',
#     'size',
#     'mime',
#     'mediatype',
#     'thumb',
    'extmetadata'
]

iiextmetadatafilter = [
#     'DateTime',
#     'DateTimeOriginal',
    'ObjectName',
    'ImageDescription',
#     'License',
#     'LicenseShortName',
#     'UsageTerms',
#     'LicenseUrl',
#     'Credit',
#     'Artist',
#     'AuthorCount',
#     'GPSLatitude',
#     'GPSLongitude',
    'Permission',
#     'Attribution',
#     'AttributionRequired',
#     'NonFree',
#     'Restrictions',
#     'DeletionReason'
]

titles = [
#     'File:Beach_veget_111029-20390_bml.jpg',
#     'File:Looking_west_along_Luce_sands_-_geograph.org.uk_-_560895.jpg',
#     'File:Dog_weight_pull.jpg',
#     'File:ZOO_%C3%9Ast%C3%AD_n_L_-_pavilon_opic_14.jpg'
#     'File:ZOO_Ústí_n_L_-_pavilon_opic_14.jpg',
#     'File:ZOO Ústí n L - pavilon opic 03.jpg'.replace(' ', '_'),
#     'File:Bundesarchiv Bild 183-1982-0623-026, Leipzig, Zoo, Affenhaus.jpg'.replace(' ', '_'),
#     'File:Cultural_Properties_and_Touring_for_Building_Numbering_in_South_Korea_(Aquarium)_(Example_2).png',
    'File:" Our Lady of Guadalupe, Xcaret Eco Park ".jpg'
]
payload = {
    'action': 'query',
    'format': 'json',
    'iiextmetadatafilter': '|'.join(iiextmetadatafilter),
    'iiextmetadatalanguage': 'en',
    'iiprop': '|'.join(iiprop),
    'maxage': '300',
    'prop': 'imageinfo',
    'smaxage': '300',
    'titles': '|'.join(titles),  # 'File:Looking_west_along_Luce_sands_-_geograph.org.uk_-_560895.jpg',
    'uselang': 'content'
}

In [6]:
res = requests.get(wikimedia_api_url, params=payload)
data = res.json()
print json.dumps(data)

{"batchcomplete": "", "query": {"pages": {"15765499": {"imagerepository": "local", "ns": 6, "pageid": 15765499, "imageinfo": [{"url": "https://upload.wikimedia.org/wikipedia/commons/c/c7/%22_Our_Lady_of_Guadalupe%2C_Xcaret_Eco_Park_%22.jpg", "extmetadata": {"ImageDescription": {"source": "commons-desc-page", "value": "Our Lady of Guadalupe, Xcaret Eco Park, Solidaridad, Quintana Roo, Mexico"}, "ObjectName": {"source": "mediawiki-metadata", "hidden": "", "value": "\" Our Lady of Guadalupe, Xcaret Eco Park \""}}, "descriptionshorturl": "https://commons.wikimedia.org/w/index.php?curid=15765499", "descriptionurl": "https://commons.wikimedia.org/wiki/File:%22_Our_Lady_of_Guadalupe,_Xcaret_Eco_Park_%22.jpg"}], "title": "File:\" Our Lady of Guadalupe, Xcaret Eco Park \".jpg"}}}}


In [7]:
# 'https://commons.wikimedia.org/w/index.php?search=clouds&title=Special%3ASearch&profile=advanced&fulltext=1&ns6=1&ns14=1'
# 'https://commons.wikimedia.org/w/index.php?search=clouds&title=Special%3ASearch&profile=advanced&fulltext=1&ns6=1&ns14=1'
# 'https://commons.wikimedia.org/w/index.php?title=Special:Search&limit=50&offset=50&ns6=1&ns14=1&search=clouds'

# '''title:Special:Search
# limit:50
# offset:50
# ns6:1
# ns14:1
# search:clouds'''

In [29]:
image_files = pd.Series()
errors_cat_ref = pd.Series()
cat_refs = Counter()


def scrape_category(cat_ref, download_image_size=256):
    '''
    cat_ref should be in the form of: u'/wiki/Category:Flora_of_Kozhikode_Beach'
    '''
    if len(image_files) % 1000 == 0:
        image_files.to_hdf('image_metadata_files.hdf', 'image_files')
        errors_cat_ref.to_hdf('image_metadata_files.hdf', 'errors_cat_ref')
        pd.Series(cat_refs).to_hdf('image_metadata_files.hdf', 'cat_refs')

    try:
        cat_refs.update([cat_ref])
        
        wikimedia_url = 'https://commons.wikimedia.org' + cat_ref

        retry_count = 0

        while retry_count < 10:
            try:
                response = requests.get(wikimedia_url)
                break
            except requests.exceptions.ConnectionError:
                retry_count += 1
                
        if retry_count >= 10:
            raise(ValueError('retries exceeded: {}'.format(cat_ref)))

        b = BeautifulSoup(response.content, 'html.parser')

        cat_trees = b.find_all('div', {'class': 'CategoryTreeSection'})

        for cat in cat_trees:
            cat_ref = cat.find('a').attrs['href']
            scrape_category(cat_ref)

        thumbs = b.find_all('div', {'class': 'thumb'})

        for thumb in thumbs:
            try:
                thumb_link = thumb.find('img').attrs['srcset'].split()[0]
            except KeyError:
                # (KeyError, "'srcset'") -> thumb is likely a video thumbnail
                continue
            except Exception as e:
                raise(e)

            # Example thumb link: https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Sulawesi_Panua_dune_trsr_DSCN0853_v1.JPG/180px-Sulawesi_Panua_dune_trsr_DSCN0853_v1.JPG
            thumb_link = thumb_link.replace('180px', '{}px'.format(download_image_size))
            thumb_name = thumb.find('img').attrs['alt']

            image_files[thumb_name] = thumb_link
            
    except Exception as e:
        error_key = '{}:{}'.format(type(e), e.__str__())
        if error_key in errors_cat_ref:
            errors_cat_ref[error_key].add(cat_ref)
        else:
            errors_cat_ref[error_key] = set([cat_ref])

In [ ]:
%%time
scrape_category('/wiki/Category:Animals')

/home/avsolatorio/ml-ai/local/lib/python2.7/site-packages/pandas/io/pytables.py:280: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->index] [items->None]

  f(store)
/home/avsolatorio/ml-ai/local/lib/python2.7/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->values] [items->None]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
/home/avsolatorio/ml-ai/local/lib/python2.7/site-packages/pandas/core/generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->values] [items->None]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [95]:
len(image_files)

20666